<a href="https://www.kaggle.com/code/ayushkhaire/crypto-dataset-daily-updates-top-1000?scriptVersionId=194084316" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json
import subprocess
import logging
import time

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,0,2020-08-01,0.000000,0.000000,0.000000,0.000000,0.000000,1.604789e+06,BTC-USD
1,1,2020-08-02,0.000000,0.000000,0.000000,0.000000,0.000000,1.930980e+05,BTC-USD
2,2,2020-08-03,0.000000,0.000000,0.000000,0.000000,0.000000,1.292250e+05,BTC-USD
3,3,2020-08-04,0.000000,0.000000,0.000000,0.000000,0.000000,1.971700e+04,BTC-USD
4,4,2020-08-05,0.000000,0.000000,0.000000,0.000000,0.000000,2.686900e+04,BTC-USD
...,...,...,...,...,...,...,...,...,...
48473,48473,2024-08-20,0.598610,0.614521,0.588849,0.594428,0.594428,1.275196e+09,LEO-USD
48474,48474,2024-08-21,0.594427,0.605304,0.586758,0.600275,0.600275,1.185129e+09,LEO-USD
48475,48475,2024-08-22,0.600277,0.603866,0.593852,0.597601,0.597601,9.436732e+08,LEO-USD
48476,48476,2024-08-23,0.597600,0.615634,0.594322,0.611239,0.611239,1.254263e+09,LEO-USD


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = 'https://finance.yahoo.com/crypto/?offset=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(40)):
    url = f'https://finance.yahoo.com/crypto/?offset={i*100}&count=25'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 40/40 [00:40<00:00,  1.02s/it]


In [6]:
symbol_list = []

for r in resp_main_list:
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('span',{'class':'symbol yf-ravs5v'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

1000


In [7]:
unique_symbol_list = []

for sy in symbol_list:
    if sy not in unique_symbol_list:
        unique_symbol_list.append(sy)

unique_symbol_list = list(set(unique_symbol_list))
print(len(unique_symbol_list))

25


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [8]:
'''

link1 = https://query1.finance.yahoo.com/v7/finance/download/TSLA?period1=1684241704&period2=1715864104&interval=1d&events=history&includeAdjustedClose=true

link2 = https://query1.finance.yahoo.com/v7/finance/download/AMD?period1=1684241769&period2=1715864169&interval=1d&events=history&includeAdjustedClose=true


here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))


period2 = current_timestamp  

period1, period2 = period2, period1

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1724642543&period2=1420070400


# create stocks directory

In [9]:
os.mkdir('crypto_data')

# Download all files

In [10]:
for stock in tqdm(symbol_list[:1000]):
    url = f"https://query1.finance.yahoo.com/v7/finance/download/{stock}?period1={period2}&period2={period1}&interval=1d&events=history&includeAdjustedClose=true"
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.csv", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.csv")
#     else:
#         print(f"Failed to download file : {stock}.csv")
#     time.sleep(3)


# https://query1.finance.yahoo.com/v7/finance/download/AMD?period1=1684244457&period2=1715866857&interval=1d&events=history&includeAdjustedClose=true

100%|██████████| 1000/1000 [07:50<00:00,  2.13it/s]


# Read a sample

In [11]:
first_file = os.listdir('/kaggle/working/crypto_data')[0]
print(first_file)
df = pd.read_csv(f'/kaggle/working/crypto_data/{first_file}')
df

NEAR-USD.csv


,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-10-14,1.694560,1.694560,1.064818,1.185806,1.185806,94692273.0
1,2020-10-15,1.177929,1.249225,1.088780,1.125557,1.125557,34933419.0
2,2020-10-16,1.124601,1.151711,0.814550,0.817511,0.817511,37020875.0
3,2020-10-17,0.817511,0.853755,0.727224,0.808136,0.808136,23630173.0
4,2020-10-18,0.809037,0.873260,0.802059,0.871674,0.871674,14376473.0
...,...,...,...,...,...,...,...
1408,2024-08-22,4.246969,4.375853,4.174996,4.301692,4.301692,164675887.0
1409,2024-08-23,4.301692,4.852125,4.301640,4.829037,4.829037,328371107.0
1410,2024-08-24,4.829037,5.247101,4.752160,5.044546,5.044546,233240361.0
1411,2024-08-25,NaN,NaN,NaN,NaN,NaN,NaN


# Work on dataset

In [12]:
csvlist = os.listdir('/kaggle/working/crypto_data')

In [13]:
megadf = pd.DataFrame()

In [14]:
for sym, file in zip(symbol_list, csvlist):
    df = pd.read_csv(f'/kaggle/working/crypto_data/{file}')
    df['Symbol'] = sym
    megadf = pd.concat([megadf, df], ignore_index=True)

In [15]:
megadf

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2020-10-14,1.694560,1.694560,1.064818,1.185806,1.185806,94692273.0,BTC-USD
1,2020-10-15,1.177929,1.249225,1.088780,1.125557,1.125557,34933419.0,BTC-USD
2,2020-10-16,1.124601,1.151711,0.814550,0.817511,0.817511,37020875.0,BTC-USD
3,2020-10-17,0.817511,0.853755,0.727224,0.808136,0.808136,23630173.0,BTC-USD
4,2020-10-18,0.809037,0.873260,0.802059,0.871674,0.871674,14376473.0,BTC-USD
...,...,...,...,...,...,...,...,...
48315,2024-08-22,2629.519043,2643.318115,2592.962891,2622.916992,2622.916992,40075181.0,DAI-USD
48316,2024-08-23,2622.916992,2792.952881,2622.380859,2763.932861,2763.932861,49233125.0,DAI-USD
48317,2024-08-24,2763.932861,2812.190918,2743.927979,2770.493896,2770.493896,54528938.0,DAI-USD
48318,2024-08-25,NaN,NaN,NaN,NaN,NaN,NaN,DAI-USD


In [16]:
first_date = megadf['Date'][0]
last_date = megadf['Date'].iloc[-1]

In [17]:
first_date

'2020-10-14'

In [18]:
megadf.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
Symbol        object
dtype: object

In [19]:
megadf['Date'] = pd.to_datetime(megadf['Date'], format="%Y-%m-%d")

In [20]:
megadf.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume              float64
Symbol               object
dtype: object

In [21]:
os.mkdir('dataset')

In [22]:
megadf.to_csv(f'dataset/1000_cryptos.csv')

# final dataset

In [23]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [24]:
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,0,2020-10-14,1.694560,1.694560,1.064818,1.185806,1.185806,94692273.0,BTC-USD
1,1,2020-10-15,1.177929,1.249225,1.088780,1.125557,1.125557,34933419.0,BTC-USD
2,2,2020-10-16,1.124601,1.151711,0.814550,0.817511,0.817511,37020875.0,BTC-USD
3,3,2020-10-17,0.817511,0.853755,0.727224,0.808136,0.808136,23630173.0,BTC-USD
4,4,2020-10-18,0.809037,0.873260,0.802059,0.871674,0.871674,14376473.0,BTC-USD
...,...,...,...,...,...,...,...,...,...
48315,48315,2024-08-22,2629.519043,2643.318115,2592.962891,2622.916992,2622.916992,40075181.0,DAI-USD
48316,48316,2024-08-23,2622.916992,2792.952881,2622.380859,2763.932861,2763.932861,49233125.0,DAI-USD
48317,48317,2024-08-24,2763.932861,2812.190918,2743.927979,2770.493896,2770.493896,54528938.0,DAI-USD
48318,48318,2024-08-25,NaN,NaN,NaN,NaN,NaN,NaN,DAI-USD


# Upload Dataset

# User secrets

In [25]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [26]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    json.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [27]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 4.00M/4.00M [00:01<00:00, 2.13MB/s]


Upload successful: 1000_cryptos.csv (4MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


# Thank you , and I will see you in next notebook :)